# Visualización 3D de Pozos - Alfa 1, Alfa 2, Alfa 3

**Proyecto:** Comparación de trayectorias de 3 pozos
**Autor:** Patricia García
**Fecha:** Enero 2025

---

Este notebook visualiza la trayectoria de 3 pozos en 3D con:
- Trayectorias completas
- Puntos de casing (13 3/8", 9 5/8", 7")
- Formaciones reservorio (Hollín Superior e Inferior)
- Nombres anónimos: Alfa 1, Alfa 2, Alfa 3

## Paso 1: Importar librerías

Necesitamos:
- `pandas` para cargar los CSV
- `plotly` para la visualización 3D
- `numpy` para cálculos

In [ ]:
# Importar librerías necesarias
import pandas as pd
import plotly.graph_objects as go
import numpy as np

print("✅ Librerías importadas correctamente")

## Paso 2: Cargar datos de los 3 pozos

Cada pozo tiene un archivo CSV con las columnas:
- `MD`: Measured Depth (profundidad medida)
- `Inc`: Inclination (inclinación)
- `Azim`: Azimuth (azimut)
- `TVD`: True Vertical Depth (profundidad vertical verdadera)
- `NS`: Desplazamiento Norte-Sur
- `EW`: Desplazamiento Este-Oeste

In [ ]:
# Cargar datos de cada pozo
alfa1 = pd.read_csv('survey_alfa1_377D.csv')  # Pozo 377D
alfa2 = pd.read_csv('survey_alfa2_378D.csv')  # Pozo 378D
alfa3 = pd.read_csv('survey_alfa3_208D.csv')  # Pozo 208D

# Ver información básica de cada pozo
print("=" * 60)
print("INFORMACIÓN DE LOS 3 POZOS")
print("=" * 60)

print(f"\nALFA 1:")
print(f"  Puntos: {len(alfa1)}")
print(f"  MD: {alfa1['MD'].min():.0f} - {alfa1['MD'].max():.0f} ft")
print(f"  TVD: {alfa1['TVD'].min():.0f} - {alfa1['TVD'].max():.0f} ft")

print(f"\nALFA 2:")
print(f"  Puntos: {len(alfa2)}")
print(f"  MD: {alfa2['MD'].min():.0f} - {alfa2['MD'].max():.0f} ft")
print(f"  TVD: {alfa2['TVD'].min():.0f} - {alfa2['TVD'].max():.0f} ft")

print(f"\nALFA 3:")
print(f"  Puntos: {len(alfa3)}")
print(f"  MD: {alfa3['MD'].min():.0f} - {alfa3['MD'].max():.0f} ft")
print(f"  TVD: {alfa3['TVD'].min():.0f} - {alfa3['TVD'].max():.0f} ft")

## Paso 3: Definir puntos importantes

Vamos a marcar:
- **Casing points**: Profundidades donde se instalaron los casings
- **Formaciones**: Hollín Superior e Inferior (reservorios principales)

In [ ]:
# CASING POINTS - Profundidades aproximadas para pozos del Oriente Ecuatoriano
# Estos son valores típicos, ajusta según tus datos reales

casing_13_38 = 2500   # Surface casing 13 3/8" (pies)
casing_9_58 = 8500    # Intermediate casing 9 5/8" (pies)
casing_7 = 10300      # Production casing 7" (pies)

print("Casing Points definidos:")
print(f"  13 3/8\": {casing_13_38} ft")
print(f"  9 5/8\": {casing_9_58} ft")
print(f"  7\": {casing_7} ft")

# FORMACIONES - Hollín Superior e Inferior
# Profundidades aproximadas (ajusta según análisis geológico)

hollin_superior = 10167  # MD aproximado (pies)
hollin_inferior = 10212  # MD aproximado (pies)

print("\nFormaciones reservorio:")
print(f"  Hollín Superior: {hollin_superior} ft")
print(f"  Hollín Inferior: {hollin_inferior} ft")

## Paso 4: Encontrar coordenadas para los marcadores

Para cada casing point y formación, necesitamos encontrar:
- La posición NS (Norte-Sur)
- La posición EW (Este-Oeste)
- La profundidad TVD

Buscamos el punto del survey más cercano a cada MD objetivo.

In [ ]:
# Función simple para encontrar punto más cercano a un MD dado
# (Usamos una función aquí porque se repite mucho, pero es muy simple)

def encontrar_punto(df, md_objetivo):
    """
    Encuentra el punto del survey más cercano a un MD objetivo.
    
    Parámetros:
    - df: DataFrame con los datos del survey
    - md_objetivo: Profundidad MD que queremos encontrar
    
    Retorna:
    - Una fila del DataFrame (el punto más cercano)
    """
    # Calcular diferencia absoluta entre cada MD y el objetivo
    diferencias = abs(df['MD'] - md_objetivo)
    
    # Encontrar índice del valor mínimo
    indice_minimo = diferencias.idxmin()
    
    # Retornar esa fila
    return df.loc[indice_minimo]

# Probar con un ejemplo
punto_ejemplo = encontrar_punto(alfa3, casing_13_38)
print(f"Ejemplo - Casing 13 3/8\" en Alfa 3:")
print(f"  MD: {punto_ejemplo['MD']:.0f} ft")
print(f"  TVD: {punto_ejemplo['TVD']:.0f} ft")
print(f"  NS: {punto_ejemplo['NS']:.1f} ft")
print(f"  EW: {punto_ejemplo['EW']:.1f} ft")

## Paso 5: Crear la figura 3D

Vamos a crear una visualización 3D interactiva con Plotly.

La figura tendrá:
- 3 trayectorias (una por cada pozo)
- Marcadores de casing
- Marcadores de formaciones Hollín

In [ ]:
# Crear figura vacía
fig = go.Figure()

print("Figura 3D creada")
print("Ahora agregaremos las trayectorias y marcadores...")

## Paso 6: Agregar trayectorias de los 3 pozos

Cada pozo se dibuja como una línea 3D con un color diferente:
- **Alfa 1**: Rojo
- **Alfa 2**: Azul
- **Alfa 3**: Verde

In [ ]:
# TRAYECTORIA ALFA 1 (Rojo)
fig.add_trace(go.Scatter3d(
    x=alfa1['NS'],           # Coordenada X (Norte-Sur)
    y=alfa1['EW'],           # Coordenada Y (Este-Oeste)
    z=-alfa1['TVD'],         # Coordenada Z (Profundidad, negativa para que vaya hacia abajo)
    mode='lines',            # Solo línea, sin marcadores
    name='ALFA 1',           # Nombre para la leyenda
    line=dict(color='red', width=6),  # Línea roja gruesa
    hovertemplate='<b>ALFA 1</b><br>MD: %{customdata:.0f} ft<extra></extra>',
    customdata=alfa1['MD']   # Mostrar MD al pasar el mouse
))

# TRAYECTORIA ALFA 2 (Azul)
fig.add_trace(go.Scatter3d(
    x=alfa2['NS'],
    y=alfa2['EW'],
    z=-alfa2['TVD'],
    mode='lines',
    name='ALFA 2',
    line=dict(color='blue', width=6),
    hovertemplate='<b>ALFA 2</b><br>MD: %{customdata:.0f} ft<extra></extra>',
    customdata=alfa2['MD']
))

# TRAYECTORIA ALFA 3 (Verde)
fig.add_trace(go.Scatter3d(
    x=alfa3['NS'],
    y=alfa3['EW'],
    z=-alfa3['TVD'],
    mode='lines',
    name='ALFA 3',
    line=dict(color='green', width=6),
    hovertemplate='<b>ALFA 3</b><br>MD: %{customdata:.0f} ft<extra></extra>',
    customdata=alfa3['MD']
))

print("✅ Trayectorias de los 3 pozos agregadas")

## Paso 7: Agregar marcadores de superficie

Marcamos el punto de superficie (0,0,0) para cada pozo.

In [ ]:
# SUPERFICIE - Origen de cada pozo
fig.add_trace(go.Scatter3d(
    x=[0, 0, 0],             # Tres pozos en el origen
    y=[0, 0, 0],
    z=[0, 0, 0],
    mode='markers+text',     # Marcadores con texto
    name='SUPERFICIE',
    marker=dict(size=12, color='gold', symbol='diamond'),
    text=['SURF', 'SURF', 'SURF'],
    textposition='top center'
))

print("✅ Marcadores de superficie agregados")

## Paso 8: Agregar casing points

Para cada pozo, marcamos los 3 puntos de casing:
- 13 3/8" (Surface)
- 9 5/8" (Intermediate)
- 7" (Production)

In [ ]:
# Lista de casings con sus propiedades
casings = [
    {'md': casing_13_38, 'nombre': '13 3/8"', 'color': 'orange'},
    {'md': casing_9_58, 'nombre': '9 5/8"', 'color': 'purple'},
    {'md': casing_7, 'nombre': '7"', 'color': 'brown'}
]

# Para cada casing, agregar marcadores en los 3 pozos
for casing in casings:
    
    # Encontrar puntos en cada pozo
    punto_alfa1 = encontrar_punto(alfa1, casing['md'])
    punto_alfa2 = encontrar_punto(alfa2, casing['md'])
    punto_alfa3 = encontrar_punto(alfa3, casing['md'])
    
    # Agregar marcadores
    fig.add_trace(go.Scatter3d(
        x=[punto_alfa1['NS'], punto_alfa2['NS'], punto_alfa3['NS']],
        y=[punto_alfa1['EW'], punto_alfa2['EW'], punto_alfa3['EW']],
        z=[-punto_alfa1['TVD'], -punto_alfa2['TVD'], -punto_alfa3['TVD']],
        mode='markers',
        name=f'Casing {casing["nombre"]}',
        marker=dict(size=10, color=casing['color'], symbol='diamond', 
                   line=dict(color='black', width=1)),
        hovertemplate=f'<b>Casing {casing["nombre"]}</b><br>MD: ~{casing["md"]:.0f} ft<extra></extra>'
    ))
    
    print(f"✅ Casing {casing['nombre']} agregado")

## Paso 9: Agregar formaciones Hollín

Marcamos Hollín Superior e Inferior en los 3 pozos.

In [ ]:
# HOLLÍN SUPERIOR
h_sup_alfa1 = encontrar_punto(alfa1, hollin_superior)
h_sup_alfa2 = encontrar_punto(alfa2, hollin_superior)
h_sup_alfa3 = encontrar_punto(alfa3, hollin_superior)

fig.add_trace(go.Scatter3d(
    x=[h_sup_alfa1['NS'], h_sup_alfa2['NS'], h_sup_alfa3['NS']],
    y=[h_sup_alfa1['EW'], h_sup_alfa2['EW'], h_sup_alfa3['EW']],
    z=[-h_sup_alfa1['TVD'], -h_sup_alfa2['TVD'], -h_sup_alfa3['TVD']],
    mode='markers',
    name='Hollín Superior',
    marker=dict(size=12, color='lime', symbol='circle',
               line=dict(color='black', width=1)),
    hovertemplate='<b>Hollín Superior</b><br>MD: ~{0:.0f} ft<extra></extra>'.format(hollin_superior)
))

# HOLLÍN INFERIOR
h_inf_alfa1 = encontrar_punto(alfa1, hollin_inferior)
h_inf_alfa2 = encontrar_punto(alfa2, hollin_inferior)
h_inf_alfa3 = encontrar_punto(alfa3, hollin_inferior)

fig.add_trace(go.Scatter3d(
    x=[h_inf_alfa1['NS'], h_inf_alfa2['NS'], h_inf_alfa3['NS']],
    y=[h_inf_alfa1['EW'], h_inf_alfa2['EW'], h_inf_alfa3['EW']],
    z=[-h_inf_alfa1['TVD'], -h_inf_alfa2['TVD'], -h_inf_alfa3['TVD']],
    mode='markers',
    name='Hollín Inferior',
    marker=dict(size=12, color='darkgreen', symbol='circle',
               line=dict(color='black', width=1)),
    hovertemplate='<b>Hollín Inferior</b><br>MD: ~{0:.0f} ft<extra></extra>'.format(hollin_inferior)
))

print("✅ Formaciones Hollín agregadas")

## Paso 10: Configurar el layout de la figura

Ajustamos títulos, ejes, tamaño de la figura y leyenda.

In [ ]:
# Configurar aspecto de la figura
fig.update_layout(
    title='<b>VISUALIZACIÓN 3D - POZOS ALFA 1, ALFA 2, ALFA 3</b>',
    scene=dict(
        xaxis_title='N-S Displacement (ft)',
        yaxis_title='E-W Displacement (ft)',
        zaxis_title='TVD (ft)',
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1.5),  # Más alto que ancho
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.2)  # Ángulo de vista inicial
        )
    ),
    height=900,              # Altura de la figura
    width=1400,              # Ancho de la figura
    showlegend=True,         # Mostrar leyenda
    legend=dict(
        x=1.02,              # Posición derecha
        y=0.98,              # Posición arriba
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor='black',
        borderwidth=1
    )
)

print("✅ Layout configurado")

## Paso 11: Mostrar la visualización

Finalmente, mostramos la figura 3D interactiva.

**Controles:**
- **Arrastrar con mouse:** Rotar la vista
- **Rueda del mouse:** Zoom in/out
- **Click en leyenda:** Mostrar/ocultar elementos
- **Hover sobre líneas/puntos:** Ver información detallada

In [ ]:
# Mostrar la figura
fig.show()

print("\n" + "="*60)
print("VISUALIZACIÓN COMPLETADA")
print("="*60)
print("\nElementos en la figura:")
print("  - 3 Trayectorias (Alfa 1, Alfa 2, Alfa 3)")
print("  - 3 Casing points por pozo (13 3/8\", 9 5/8\", 7\")")
print("  - 2 Formaciones (Hollín Superior e Inferior)")
print("  - Marcadores de superficie")
print("\nUsa el mouse para explorar la visualización 3D!")

## Paso 12 (Opcional): Guardar la figura como HTML

Si quieres guardar la visualización para compartirla o verla más tarde.

In [ ]:
# Guardar como archivo HTML
fig.write_html('visualizacion_3_pozos_alfa.html')

print("✅ Figura guardada como 'visualizacion_3_pozos_alfa.html'")
print("   Puedes abrir este archivo en cualquier navegador")